# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [19]:
rdd = sc.textFile('enron_mails_small.csv').cache()
def mapper(pid, rows):
    if pid == 0:
        next(rows)
    import csv
    for fields in csv.reader(rows):
        email_from = fields[1]
        if email_from[email_from.find('@'):] == '@enron.com':
            emails_to_str = fields[2].split(';')
            for e_to in emails_to_str:
                yield (email_from[:email_from.find('@')], e_to[:e_to.find('@')])
    
    
reciprocalList = rdd.mapPartitionsWithIndex(mapper)\
    .map(lambda x:(x,0) ) \
    .groupByKey() \
    .map(lambda x: (tuple(sorted(x[0])),1)) \
    .reduceByKey(lambda v1 , v2: v1 + v2) \
    .filter(lambda x: x[1]>=2) \
    .sortByKey() \
    .map(lambda x : ('reciprocal', x[0][0].replace(".", " ").title(), x[0][1].replace(".", " ").title())) \
    .collect()

len(reciprocalList)
reciprocalList

[('reciprocal', 'Brenda Whitehead', 'Elizabeth Sager'),
 ('reciprocal', 'Carol Clair', 'Debra Perlingiere'),
 ('reciprocal', 'Carol Clair', 'Mark Taylor'),
 ('reciprocal', 'Carol Clair', 'Richard Sanders'),
 ('reciprocal', 'Carol Clair', 'Sara Shackleton'),
 ('reciprocal', 'Carol Clair', 'Tana Jones'),
 ('reciprocal', 'Debra Perlingiere', 'Kevin Ruscitti'),
 ('reciprocal', 'Drew Fossum', 'Susan Scott'),
 ('reciprocal', 'Elizabeth Sager', 'Janette Elbertson'),
 ('reciprocal', 'Elizabeth Sager', 'Mark Haedicke'),
 ('reciprocal', 'Elizabeth Sager', 'Mark Taylor'),
 ('reciprocal', 'Elizabeth Sager', 'Richard Sanders'),
 ('reciprocal', 'Eric Bass', 'Susan Scott'),
 ('reciprocal', 'Fletcher Sturm', 'Greg Whalley'),
 ('reciprocal', 'Fletcher Sturm', 'Sally Beck'),
 ('reciprocal', 'Gerald Nemec', 'Susan Scott'),
 ('reciprocal', 'Grant Masson', 'Vince Kaminski'),
 ('reciprocal', 'Greg Whalley', 'Richard Sanders'),
 ('reciprocal', 'Janette Elbertson', 'Mark Taylor'),
 ('reciprocal', 'Janette Elb

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [20]:
SAT_FN='SAT_Results.csv'
HSD_FN='DOE_High_School_Directory_2014-2015.csv'

In [21]:
dfSatScores = spark.read.load(SAT_FN, format='csv', header = True, inferSchema=True)
dfSatScores =dfSatScores.select('DBN', dfSatScores['Num of SAT Test Takers'].cast('int').alias('ntakers'),
                          dfSatScores['`SAT Math Avg. Score`'].cast('int').alias('score')).na.drop()

dfSatScores = dfSatScores.select('DBN', 'ntakers', (dfSatScores.score*dfSatScores.ntakers).alias('total'))

dfSatScores.show(10)

+------+-------+------+
|   DBN|ntakers| total|
+------+-------+------+
|02M047|     16|  6400|
|21K410|    475|207575|
|30Q301|     98| 43120|
|17K382|     59| 22066|
|18K637|     35| 13335|
|32K403|     50| 18300|
|09X365|     54| 18306|
|11X270|     56| 22064|
|05M367|     33| 12078|
|14K404|     68| 24276|
+------+-------+------+
only showing top 10 rows



In [22]:
import csv

schools = sc.textFile(HSD_FN).cache()

def extractBus(pid, rows):
    if pid == 0:
        next(rows)

    for fields in csv.reader(rows):
        if len(fields)==58:
            for bus in fields[10].split(","):
                yield (fields[0], bus.strip())
    

def extractSubway(pid, rows):
    if pid == 0:
        next(rows)

    for fields in csv.reader(rows):
        if len(fields)==58:
            for bus in fields[11].split(";"):
                yield (fields[0], bus.strip())
        

schoolBus = schools.mapPartitionsWithIndex(extractBus) \
                .collect()
                    
schoolSubway= schools.mapPartitionsWithIndex(extractSubway) \
                .collect()

dfSchoolBus= sqlContext.createDataFrame(schoolBus, ['DBN', 'Bus'])
dfSchoolSubway = sqlContext.createDataFrame(schoolSubway, ['DBN', 'Subway'])

dfSchoolBus.show(10)
dfSchoolSubway.show(10)

+------+-------+
|   DBN|    Bus|
+------+-------+
|01M292|    B39|
|01M292|   M14A|
|01M292|   M14D|
|01M292|    M15|
|01M292|M15-SBS|
|01M292|    M21|
|01M292|    M22|
|01M292|     M9|
|01M448|   M14A|
|01M448|   M14D|
+------+-------+
only showing top 10 rows

+------+--------------------+
|   DBN|              Subway|
+------+--------------------+
|01M292|    B, D to Grand St|
|01M292|  F to East Broadway|
|01M292|J, M, Z to Delanc...|
|01M448|  F to East Broadway|
|01M448|J, M, Z to Delanc...|
|01M450|    6 to Astor Place|
|01M450|        L to 1st Ave|
|01M509|    B, D to Grand St|
|01M509|F, J, M, Z to Del...|
|01M539|F, J, M, Z to Del...|
+------+--------------------+
only showing top 10 rows



In [23]:
#find ave score for by bus
dfBusAgeResults = dfSchoolBus.join(dfSatScores, dfSchoolBus.DBN==dfSatScores.DBN, how='inner')
dfBusAgeResults = dfBusAgeResults.groupBy('Bus').sum('ntakers', 'total')
dfBusAgeResults = dfBusAgeResults.withColumn('avg', (dfBusAgeResults[2]/dfBusAgeResults[1]).cast('int')) \
            .select('Bus', 'avg')
dfBusAgeResults.show()

+-------+---+
|    Bus|avg|
+-------+---+
|   Bx33|426|
|   Q101|464|
|    Q65|464|
|B44-SBS|465|
|     Q7|407|
|   Q110|462|
|    B39|438|
|    Q32|466|
|   Q100|454|
|    Q16|445|
|   M104|477|
|    Q50|392|
|    Q42|582|
|   M103|441|
|    Q28|492|
|     M4|442|
|    M66|518|
|     M7|483|
|    Bx7|402|
|     B4|466|
+-------+---+
only showing top 20 rows



In [24]:
#find ave score for by Subway
dfSubwayAgeResults = dfSchoolSubway.join(dfSatScores, dfSchoolSubway.DBN==dfSatScores.DBN, how='inner')
dfSubwayAgeResults = dfSubwayAgeResults.groupBy('Subway').sum('ntakers', 'total')
dfSubwayAgeResults = dfSubwayAgeResults.withColumn('avg', (dfSubwayAgeResults[2]/dfSubwayAgeResults[1]).cast('int'))\
            .select('Subway', 'avg')
dfSubwayAgeResults.show()

+--------------------+---+
|              Subway|avg|
+--------------------+---+
|4 to 138th St - G...|378|
|       D to 25th Ave|382|
|4, 5, 6, N, Q, R ...|451|
|      Z to Marcy Ave|355|
|7 to Rawson St-33...|482|
|4, 5, 6, L, N, Q,...|391|
|   C, E to Spring St|452|
|  A, C to Euclid Ave|414|
|1 to 137th St - C...|500|
|2, 3, 4, 5, F, S ...|393|
|6 to 138th St-3rd...|371|
|4, B, D to Kingsb...|372|
|   C to Franklin Ave|364|
|2, 5 to Intervale Av|432|
|  C to Lafayette Ave|635|
|4 to 149th St-Gra...|400|
|    2, 5 to 174th St|379|
|       1 to 157th St|376|
|    C to Liberty Ave|369|
|A, S to Beach 98t...|419|
+--------------------+---+
only showing top 20 rows

